<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/10-Comparacion-clasificadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/DCDPUAEM/DCDP/raw/main/03%20Machine%20Learning/img/vs.jpeg">

# Comparación de varios clasificadores

En esta notebook compararemos el rendimiento y tiempo de ejecución de varios algoritmos de clasificación.

* `KNeighborsClassifier`: Clasifica comparando una instancia de acuerdo a las etiquetas de los vecinos más cercanos en el espacio de features.

* `AdaBoostClassifier`: Es un meta-algoritmo que crea un ensamble de clasificadores con una técnica diferente al *bagging*.

* `BaggingClassifier`: Es un meta-algoritmo que crea un ensamble de clasificadores usando *bagging*.

* `GaussianNB` (Bayes Ingenuo Gaussiano): Se clasifican las instancias calculando la probabilidad de que cada instancia pertenezca a cada clase, dadas sus features, bajo la suposición de que las features siguen una distribución normal y son independientes entre sí dadas la clase (esta última es la suposición *naive*).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import time
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=1001)

In [ ]:
clfs = [DecisionTreeClassifier(),RandomForestClassifier(),
        SVC(),LogisticRegression(),
        GaussianNB(),KNeighborsClassifier(),
        AdaBoostClassifier(estimator=SVC(),algorithm='SAMME'),
        AdaBoostClassifier(estimator=GaussianNB(),algorithm='SAMME'),
        BaggingClassifier(estimator=SVC())]

names = [x.__class__.__name__ for x in clfs]  # Obtener el nombre de cada clasificador, como <string>
names[-3] += '_SVC'
names[-2] += '_GNB'
names[-1] += '_SVC'
times = []
accs = []

for clf in clfs:
    pl = Pipeline([('scl',StandardScaler()),
                    ('clf',clf)])
    start = time.time()
    pl.fit(X_train,y_train)
    end = time.time()
    times.append(end-start)
    y_pred = pl.predict(X_test)
    accs.append(accuracy_score(y_test,y_pred))

resultados_df = pd.DataFrame(data={'algoritmo': names,
                   'accuracy': accs,
                   'duración': times})

## Rendimiento

In [ ]:
resultados_df.sort_values(by='accuracy',ascending=False)

## Tiempo de ejecución

In [ ]:
resultados_df.sort_values(by='duración',ascending=True)

A continuación se presenta la complejidad big-O de algunos algoritmos:

* Naive Bayes: $O(Nd)$
* k-NN: $O(1)$ (space complexity of training is $O(Nd)$ since the data needs to be stored, which also takes time).
* SVM: $O(N^2)$ or $O(N^3)$ depending on the kernel.
* DecisionTree: $O(n\log n)$